In [13]:
import pandas as pd
import os

def read_csvs(path):
    dfs = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            df = pd.read_csv(path + file)
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


In [14]:
df = read_csvs(path = "/home/r4ph/desenv/phd/exception-miner/output/fixes/")

In [15]:
len(df.file.unique())

729

In [17]:
df_rq5 = df.agg({'file': ['count'], 'function': ['count'], 'n_try_pass': ['sum'], 'n_generic_except': ['sum'], 'n_nested_try': ['sum'], 
                 'n_bare_except': ['sum'], 'n_captures_misplaced_bare_raise': ['sum'], 'n_captures_broad_raise' : ['sum'], 'n_bare_raise_finally' : ['sum'] })

In [18]:
df_rq5

,file,function,n_try_pass,n_generic_except,n_nested_try,n_bare_except,n_captures_misplaced_bare_raise,n_captures_broad_raise,n_bare_raise_finally
count,24044.0,24044.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sum,NaN,NaN,529.0,642.0,40.0,150.0,9.0,397.0,0.0


In [19]:
df_rq5

,file,function,n_try_pass,n_generic_except,n_nested_try,n_bare_except,n_captures_misplaced_bare_raise,n_captures_broad_raise,n_bare_raise_finally
count,24044.0,24044.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sum,NaN,NaN,529.0,642.0,40.0,150.0,9.0,397.0,0.0
